In [64]:
from neon.models import Model
from neon.data import ArrayIterator
from neon.backends import gen_backend
from hsa import emu_connect
import importlib
import socket
import pandas
import numpy as np
import random

In [ ]:
# hack to import modules in pycharm and browser
import sys
sys.path.append("../")

In [43]:
gen_backend(backend='cpu', batch_size=128).bsz

128

In [81]:
prim_soc = socket.create_connection(("localhost", 9090))
prim_soc

<socket.socket fd=36, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 43570), raddr=('127.0.0.1', 9090)>

In [82]:
importlib.reload(emu_connect)
emu = emu_connect.Emu2(prim_soc)

In [80]:
emu.close()

'true\n'

In [46]:
nn = Model("mimi_mario_1_1_first.prm")

In [36]:
emu.load_slot(10)
emu.step()
example_ram = emu.get_ram()
example_ram_df = pandas.DataFrame.from_records([example_ram])

In [38]:
example_ram_df

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,79,208,1,0,40,32,0,5,1,...,0,0,4,0,1,0,0,0,0,165


In [73]:
np.random.uniform(0, 1, 8)

array([ 0.60774431,  0.77031497,  0.94191602,  0.37049298,  0.38388568,
        0.98316719,  0.10849471,  0.93343757])

In [44]:
gen_backend(backend='cpu', batch_size=1)
example_ram_Neon= ArrayIterator(example_ram_df)

In [48]:
example_input_neon = nn.get_outputs(example_ram_Neon)

In [75]:
# A,B,down,left,right,select,start,up
input_transform_keys = ["A", "B", "down", "left", "right", "select", "start",
                        "up"]
def neon_input_to_emu(neon_input:np.ndarray):
    # not schour if wheighted random is a good joice
    input_booleans = neon_input> np.random.uniform(0, 1, 8)
    numpy_values = [r[0] for r in input_booleans.T]
    return  dict(zip(input_transform_keys, numpy_values))
neon_input_to_emu(example_input_neon)

{'A': False,
 'B': True,
 'down': False,
 'left': False,
 'right': True,
 'select': False,
 'start': False,
 'up': False}

In [77]:
def run_simulation(emu):
    # sanity hack, to make should nothing is left in the buffer
    # prim_soc.recv(2**10)
    emu.load_slot(10)
    emu.speed_mode("normal")
    for i in range(60 * 60):
        emu.step()
        ram = emu.get_ram()
        neon_input = nn.get_outputs(ArrayIterator(pandas.DataFrame.from_records([ram])))
        emu.input_write(neon_input_to_emu(neon_input))

In [83]:
run_simulation(emu)

In [84]:
for i in range(2048):
    prim_soc.recv(1)